In [1]:
infile = open ("data/preprocessed.txt",'r',encoding = 'ISO-8859-1')

In [2]:
actor_movies = []
for line in infile:
    actor_movies.append(line[:-1].split("\t\t")) 

In [3]:
print (len(actor_movies))

113132


In [4]:
infile.close()

In [5]:
# Construct the movie dictionary
actDict = {}
movieDict = {}
for e in actor_movies :
    actId = e[0]
    actDict[actId] = e[1:]
    for movieName in e[1:] :
        #num += 1
        movieName.strip(" ")
        movieName.strip("\t")
        #print ("%d %s"%(num,movieName))
        if movieName in movieDict and movieName :
            movieDict[movieName].append(actId)
        else:
            movieDict[movieName] = [len(movieDict)] #movieId
            movieDict[movieName].append(actId)

In [6]:
print (len(actDict))
print (len(movieDict))

113132
468201


In [9]:
movieDict_new = {}
#mId = 0
for k,aList in movieDict.items():
    if len(aList) > 5:
        #aList[0] = mId
        movieDict_new[k] = aList
        #mId += 1

In [10]:
print (len(movieDict_new))
print (movieDict_new['Xi chun tu (1959)'])

203576
[342114, '39228', '40639', '70728', '73273', '74152', '89708', '94652', '106235']


In [10]:
del movieDict

#### Generate the movie-id map file 

In [11]:
movie_id_file = open ("data/movie_id_map.txt",'w',encoding = 'ISO-8859-1')
for movie in movieDict_new:
    s = movie + "\t" + str(movieDict_new[movie][0]) +"\n"
    movie_id_file.write(s)
movie_id_file.close()

#### Use the hash table to calculate the number of intersections and unions 

In [12]:
inter={}
union={}
for key_movie in movieDict_new: # select a key movie
    #print ("key_movie:%s"%key_movie)
    for i in range(1,len(movieDict_new[key_movie])): # find all the actors in this key movie
        #print (actDict[aId])
        actId =  movieDict_new[key_movie][i]
        for movie in actDict[actId]: # find all the other movies that the actor has acted
            #print (movie)
            if movie==key_movie or (movie not in movieDict_new):
                continue#skip same movie or no movie or movie ID
            totActor_num = len(movieDict_new[key_movie])+len(movieDict_new[movie])-2
            #print ("%s\t%s\n"%(key_movie,movie))
            if (movieDict_new[key_movie][0],movieDict_new[movie][0]) in inter:
                inter[(movieDict_new[key_movie][0],movieDict_new[movie][0])]+=1
            elif ((movieDict_new[movie][0], movieDict_new[key_movie][0])) in inter:
                inter[(movieDict_new[movie][0], movieDict_new[key_movie][0])]+=1
            else:
                inter[(movieDict_new[key_movie][0],movieDict_new[movie][0])]=1
                union[(movieDict_new[key_movie][0],movieDict_new[movie][0])]=totActor_num

print ('inter:%d'%(len(inter)))

inter:66518294


### Write edgelist file 

In [13]:
outfile = open("data/movie_graph.txt",'w',encoding = 'ISO-8859-1')

In [14]:
for key_edge in inter:
    #print (key_edge)
    numerator = inter[key_edge]/2
    denominator = union[key_edge]-numerator
    weight = float(numerator)/float(denominator)
    #print (weight)
    s = '%d\t%d\t%5f\n'%(key_edge[0],key_edge[1],weight)
    #print (s)
    outfile.write(s)
print ("Graph has been constructed successfully!")

Graph has been constructed successfully!


In [15]:
outfile.close()

In [17]:
print (inter[(0,1)])

2


### Generate movieid-genre file 

In [ ]:
# File Format: movieName\tmovieId
mapfile = open ("data/movie_id_map.txt",'r',encoding = 'ISO-8859-1')

movieNameDict = {}
for line in mapfile:
    movieNameDict[line[:-1].split("\t")[0]] = [eval(line[:-1].split("\t")[1])]

movieNameDict['American Pimp (1999)']

mapfile.close()

In [ ]:
# File Format: movieName\t\tmoiveGenre
genrefile = open ("data/movie_genre.txt",'r',encoding = 'ISO-8859-1')
num = 0
for genre in genrefile:
    Name_Genre = genre[:-1].split("\t\t")
    #num += 1
    #print (Name_Genre[0])
    #print (Name_Genre[1])
    if Name_Genre[0] in movieNameDict:
        movieNameDict[Name_Genre[0]].append(Name_Genre[1])
    else:
        #print (Name_Genre[0])
        num += 1 

In [ ]:
movieid_genre_file = open ("data/movieid_genre.txt",'w',encoding = 'ISO-8859-1')
for movie in movieNameDict:
    if len(movieNameDict[movie]) == 2:
        movieid_genre_file.write('%d\t%s\n'%(movieNameDict[movie][0],movieNameDict[movie][1]))
movieid_genre_file.close()

# 179046 Effect Movies(have genre)